# Back Propagation

In [1]:
import random
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error as mse
from matplotlib import pyplot as plt

## 1. loading of data

In [2]:
import pickle
with open('data.pkl', 'rb') as f:
    data = pickle.load(f)
print(data.shape)
X = data[:, :5]
y = data[:, -1]
print(X.shape, y.shape)

(506, 6)
(506, 5) (506,)


In [3]:
np.array(X)

array([[-1.2879095 , -0.12001342, -1.45900038, -0.66660821, -0.14421743],
       [-0.59338101,  0.36716642, -0.30309415, -0.98732948, -0.74026221],
       [-0.59338101, -0.26581176, -0.30309415, -0.98732948, -0.74026221],
       ...,
       [ 0.11573841,  0.79744934,  1.17646583, -0.80321172,  0.15812412],
       [ 0.11573841,  0.73699637,  1.17646583, -0.80321172,  0.15812412],
       [ 0.11573841,  0.43473151,  1.17646583, -0.80321172,  0.15812412]])

In [4]:
seed = 3

# 2. Computational graph

<img src='https://i.imgur.com/seSGbNS.png'>

<pre>
1. if you observe the graph, we are having input features [f1, f2, f3, f4, f5] and 9 weights [w1, w2, w3, w4, w5, w6,    w7, w8, w9]
2. the final output of this graph is a value L which is computed as (Y-Y')^2
</pre>

### Task 1: Implementing backpropagation and Gradient checking


<pre>1. <b>Check this video for better understanding of the computational graphs and back propagation:</b> <a href='https://www.youtube.com/watch?v=i94OvYb6noo#t=1m33s'>https://www.youtube.com/watch?v=i94OvYb6noo</a>
</pre>

<pre>
2. <b>write two functions</b>

#you can modify the definition of this function according to your needs
<font color='green'>
def forward_propagation(X, y, W):
        <font color='grey'>
        # X: input data point, note that in this assignment you are having 5-d data points
        # y: output varible
        # W: weight array, its of length 9, W[0] corresponds to w1 in graph, W[1] corresponds to w2 in graph, ..., W[8] corresponds to w9 in graph.
        # write code to compute the value of L=(y-y')^2
        </font>
        return (L, any other variables which you might need to use for back propagation)
        <font color='grey'>
        # Hint: you can use dict type to store the required intermediate variables 
        </font>
</font>
</pre>

<pre>
# you can modify the definition of this function according to your needs
<font color='blue'>
def backward_propagation(L, Variables):
        <font color='grey'>
        # L: the loss we calculated for the current point
        # Variables: the outputs of the forward_propagation() function
        # write code to compute the gradients of each weight [w1,w2,w3,...,w9]
        </font>
        return dW
        <font color='grey'>
        # here dW can be a list, or dict or any other data type wich will have gradients of all the weights
        # Hint: you can use dict type to store the required variables 
        </font>
</font>
</pre>
3. <b> <a href='https://towardsdatascience.com/how-to-debug-a-neural-network-with-gradient-checking-41deec0357a9'>Gradient checking</a></b>:<a href='https://towardsdatascience.com/how-to-debug-a-neural-network-with-gradient-checking-41deec0357a9'>blog link</a> 

<pre>we know that the derivative of any function is </pre>$$\lim_{\epsilon\to0}\frac{f(x+\epsilon)-f(x-\epsilon)}{2\epsilon}$$
<pre>
The definition above can be used as a numerical approximation of the derivative. Taking an epsilon small enough, the calculated approximation will have an error in the range of epsilon squared. 

In other words, if epsilon is 0.001, the approximation will be off by 0.00001.

Therefore, we can use this to approximate the gradient, and in turn make sure that backpropagation is implemented properly. This forms the basis of gradient checking!

</pre>

<font >
lets understand the concept with a simple example:
$f(w1,w2,x1,x2)=w_{1}^{2} . x_{1} + w_{2} . x_{2}$ 

from the above function lets assume $w_{1}=1$, $w_{2}=2$, $x_{1}=3$, $x_{2}=4$ the gradient of $f$ w.r.t $w_{1}$ is

\begin{array} {lcl}
\frac{df}{dw_{1}} = dw_{1} &=&2.w_{1}.x_{1} \\& = &2.1.3\\& = &6
\end{array}


let calculate the aproximate gradient of $w_{1}$ as mentinoned in the above formula and considering $\epsilon=0.0001$

\begin{array} {lcl}
dw_1^{approx} & = & \frac{f(w1+\epsilon,w2,x1,x2)-f(w1-\epsilon,w2,x1,x2)}{2\epsilon} \\ & = & \frac{((1+0.0001)^{2} . 3 + 2 . 4) - ((1-0.0001)^{2} . 3 + 2 . 4)}{2\epsilon} \\ & = & \frac{(1.00020001 . 3 + 2 . 4) - (0.99980001. 3 + 2 . 4)}{2*0.0001} \\ & = & \frac{(11.00060003) - (10.99940003)}{0.0002}\\ & = & 5.99999999999
\end{array}

Then, we apply the following formula for gradient check: <i>gradient_check</i> = 
$\frac{\left\Vert\left (dW-dW^{approx}\rm\right) \right\Vert_2}{\left\Vert\left (dW\rm\right) \right\Vert_2+\left\Vert\left (dW^{approx}\rm\right) \right\Vert_2}$

The equation above is basically the Euclidean distance normalized by the sum of the norm of the vectors. We use normalization in case that one of the vectors is very small.
As a value for epsilon, we usually opt for 1e-7. Therefore, if gradient check return a value less than 1e-7, then it means that backpropagation was implemented correctly. Otherwise, there is potentially a mistake in your implementation. If the value exceeds 1e-3, then you are sure that the code is not correct.

in our example: <i>gradient_check</i> $ = \frac{(6 - 5.999999999994898)}{(6 + 5.999999999994898)} = 4.2514140356330737e^{-13}$

you can mathamatically derive the same thing like this

\begin{array} {lcl}
dw_1^{approx} & = & \frac{f(w1+\epsilon,w2,x1,x2)-f(w1-\epsilon,w2,x1,x2)}{2\epsilon} \\ & = & \frac{((w_{1}+\epsilon)^{2} . x_{1} + w_{2} . x_{2}) - ((w_{1}-\epsilon)^{2} . x_{1} + w_{2} . x_{2})}{2\epsilon} \\ & = & \frac{4. \epsilon.w_{1}. x_{1}}{2\epsilon} \\ & = &  2.w_{1}.x_{1}
\end{array}

to do this task you need to write a function 
<pre>
<font color='darkblue'>
W = initilize_randomly
def gradient_checking(data_point, W):
    <font color='grey'>
    # compute the L value using forward_propagation()
    # compute the gradients of W using backword_propagation()
    </font>
    approx_gradients = []
    for each wi weight value in W:
        <font color='grey'>
        # add a small value to weight wi, and then find the values of L with the updated weights
        # subtract a small value to weight wi, and then find the values of L with the updated weights
        # compute the approximation gradients of weight wi
        </font>
        approx_gradients.append(approximation gradients of weight wi)
    <font color='grey'>
    # compare the gradient of weights W from backword_propagation() with the aproximation gradients of weights with      gradient_check formula
    </font>
    return gradient_check
</font>
NOTE: you can do sanity check by checking all the return values of gradient_checking(), they have to be zero. if not you have bug in your code
</pre>

In [ ]:
class OurNeuralNetwork:
  '''
  A neural network with:
    - 2 inputs
    - a hidden layer with 2 neurons (h1, h2)
    - an output layer with 1 neuron (o1)
  *** DISCLAIMER ***:
  The code below is intended to be simple and educational, NOT optimal.
  Real neural net code looks nothing like this. DO NOT use this code.
  Instead, read/run it to understand how this specific network works.
  '''
  def __init__(self):
    # Weights
    self.w1 = np.random.normal()
    self.w2 = np.random.normal()
    self.w3 = np.random.normal()
    self.w4 = np.random.normal()
    self.w5 = np.random.normal()
    self.w6 = np.random.normal()

    # Biases
    self.b1 = np.random.normal()
    self.b2 = np.random.normal()
    self.b3 = np.random.normal()

In [5]:
def initialize_weights(n):
    """
    Initialize weights.
    
    Weight matrices will be initialized to random values from uniform normal
    distribution.
    

    Parameters
    ----------
    n : int
         Size of weight vector


    Returns
    ------- 
    W : array_like
        A (9 x 1) matrix of weights
    """
    
    random.seed(seed)
    W = np.array([random.uniform(0,1) for i in range(n)])
    
    return W
    
W = initialize_weights(9)

In [163]:
# Class writing style
# https://github.com/KirillShmilovich/MLP-Neural-Network-From-Scratch/blob/master/MLP.ipynb
def sigmoid(x):
    """
    
    Compute the sigmoid of `x`, calculated element-wise
    sigmoid applied to `x` element-wise
    
    
    Parameters
    ----------
    x : float or array_like input


    Returns
    -------
    sigmoid(x) : float or array_like
    """
    return 1/(1+np.exp(-x))

In [164]:
def forward_propagation(F, y, W):
    """
    Calculate loss, y_pred and intermediate variable dictionary
    
    
    Parameters
    ----------
    F : array_like input for input features
    y : array_like input for target labels
    W : array_like input for weights
    
    
    Returns
    -------
    y_pred : float prediction
    Loss : squared loss
    var_dict : dictionary for intermediate variables
    """
    
    q1 = W[0] * F[0]
    q2 = W[1] * F[1]
    q3 = q1 + q2
    q4 = q1 + q2
    q5 = q3 * q4
    q6 = W[5] + q5
    q7 = np.exp(q6)
    q8 = W[6] + q7
    q9 = np.tanh(q8)
    q10 = W[2] * F[2]
    q11 = np.sin(q10)
    q12 = W[3] * F[3]
    q13 = W[4] * F[4]
    q14 = q12 + q13
    q15 = q11 * q14
    q16 = W[7] + q15
    q17 = sigmoid(q16)
    q18 = W[8] * q17
    y_pred = q9 + q18
    Loss = (y - y_pred)**2
    var_dict = {
        'q1':q1,'q2':q2,'q3':q3,'q4':q4,'q5':q5,'q6':q6,'q7':q7,'q8':q8,'q9':q9,'q10':q10,'q11':q11,'q12':q12,'q13':q13,'q14':q14,'q15':q15,'q16':q16,'q17':q17,'q18':q18
    }
    
    return y_pred, Loss, var_dict

In [233]:
def backward_propagation(X, y, W, vd):
    """
    Calculating derivatives and storing frequently used derivatives using memoization
    
    
    Parameters
    ----------
    X : array_like input for input features
    y : array_like input for target labels
    W : array_like input for weights
    vd : dictionry with keys from q1 to q18
    
    
    Returns
    -------
    derivatives_list : list with derivatives of w1 to w9
    """
    
    y_pred = vd['q9'] + vd['q18']
    dL_dy = -2*(y-y_pred)
    
    
    dqy_q18 = 1
    dqy_q9 = 1
    dq1 = X[0]
    dq2 = X[1]
    dq3 = 1
    dq4 = 1
    dq5 = 2 * vd['q3']
    dq6 = 1
    dq7 = vd['q7']
    dq8 = 1
    dq9 = 1 - np.tanh(vd['q8'])**2
    dq10 = X[2]
    dq11 = np.cos(vd['q10'])
    dq12 = X[3]
    dq13 = X[4]
    dq14 = 1
    dq15_q11 = vd['q14']
    dq15_q14 = vd['q11']
    dq16 = 1
    dq17 = vd['q17']*(1 - vd['q17'])
    dq18_w9 = vd['q17']
    dq18_q17 = W[8]


    dw1 = dqy_q9*dq9*dq8*dq7*dq6*dq5*dq3*dq1
    dw2 = dqy_q9*dq9*dq8*dq7*dq6*dq5*dq4*dq2
    dw3 = dqy_q18*dq18_q17*dq17*dq16*dq15_q11*dq11*dq10
    dw4 = dqy_q18*dq18_q17*dq17*dq16*dq15_q14*dq14*dq12
    dw5 = dqy_q18*dq18_q17*dq17*dq16*dq15_q14*dq14*dq13
    dw6 = dqy_q9*dq9*dq8*dq7*dq6
    dw7 = dqy_q9*dq9*dq8
    dw8 = dqy_q18*dq18_q17*dq17*dq16
    dw9 = dqy_q18*dq18_w9
    
    
    grad = np.array([dw1, dw2, dw3, dw4, dw5, dw6, dw7, dw8, dw9])
    
    return grad

In [246]:
#Source -  https://towardsdatascience.com/how-to-debug-a-neural-network-with-gradient-checking-41deec0357a9
def gradient_check(X, y, W, e=1e-7):
    y_pred, Loss, var_dict = forward_propagation(X, y, W)
    grad = backward_propagation_2(X, y, W, var_dict)
    approx_gradients = []
    
    for i, weight in enumerate(W):
        w_plus, w_minus = W.copy(), W.copy()
        w_plus[i] = w_plus[i] + e
        w_minus[i] = w_minus[i] - e
        y_pred_p, _, _ = forward_propagation(X, y, w_plus)
        y_pred_n, _, _ = forward_propagation(X, y, w_minus)
        grad_approx = (y_pred_p-y_pred_n)/(2*e)
        approx_gradients.append(grad_approx)
        
        numerator = np.linalg.norm(grad[i] - approx_gradients[i])
        denominator = np.linalg.norm(grad[i]) + np.linalg.norm(approx_gradients[i])
        difference = numerator / denominator 
        
        if difference <= 1e-7:
            print('weight {0} is correct'.format(i+1))
        else:
            print('weight {0} is in-correct'.format(i+1))
        

In [247]:
gradient_check(X[0], y[0], W)

weight 1 is correct
weight 2 is correct
weight 3 is correct
weight 4 is correct
weight 5 is correct
weight 6 is correct
weight 7 is correct
weight 8 is correct
weight 9 is correct


### Task 2: Optimizers

1. As a part of this task, you will be implementing 3 type of optimizers(methods to update weight)
2. check this video and blog: https://www.youtube.com/watch?v=gYpoJMlgyXA,  http://cs231n.github.io/neural-networks-3/
3. use the same computational graph that was mentioned above to do this task
4. initilze the 9 weights from normal distribution with mean=0 and std=0.01

5. 

<pre>
    for each epoch(1-100):
        for each data point in your data:
            using the functions forward_propagation() and backword_propagation() compute the gradients of weights
            update the weigts with help of gradients  ex: w1 = w1-learning_rate*dw1
</pre>

6.

<pre>
<b>task 2.1</b>: you will be implementing the above algorithm with <b>Vanilla update</b> of weights
<b>task 2.2</b>: you will be implementing the above algorithm with <b>Momentum update</b> of weights
<b>task 2.3</b>: you will be implementing the above algorithm with <b>Adam update</b> of weights
</pre>



In [248]:
def SGD():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-248-1c6f72b23f17>, line 1)

In [ ]:

        
def momentum(x, y):
    return w

def adam(x, y):
    return w